In [42]:
import requests
import pandas as pd

def getProtocols():
    resp = requests.get('https://api.llama.fi/protocols')
    return resp.json()

protocols = getProtocols()
df = pd.json_normalize(protocols)

In [43]:
def getProtocolTVLHistorical(protocol_slug):
    url = f'https://api.llama.fi/protocol/{protocol_slug}'
    resp = requests.get(url)
    if resp.status_code == 200:
        resp = resp.json()
        # print(list(resp.keys()))
        # print(type(resp['tvl']))
        # print(resp['tvl'][0])  # 
        temp_dates, temp_tvls = [], []
        for dict in resp['tvl']:
            temp_dates.append(dict['date'])
            temp_tvls.append(dict['totalLiquidityUSD'])
        return temp_dates, temp_tvls
    
def processTVLData(tdates, ttvls):
    tdf = pd.DataFrame(data={'timestamp': tdates, 'tvls': ttvls})
    tdf['dates'] = tdf['timestamp'].values.astype(dtype='datetime64[s]')
    tdf['dates'] = pd.to_datetime(tdf['dates']).dt.date
    tdf['7dma'] = tdf['tvls'].rolling(window=7, min_periods=1).mean()
    tdf['7dma_%'] = tdf['7dma'].pct_change(periods=1).fillna(0)
    tdf['1mma'] = tdf['tvls'].rolling(window=30, min_periods=1).mean()
    tdf['1mma_%'] = tdf['1mma'].pct_change(periods=1).fillna(0)
    sevenday = tdf['7dma_%'].values.tolist()[-1]
    thirtyday = tdf['1mma_%'].values.tolist()[-1]
    return sevenday, thirtyday


min_tvl = 10_000_000
print(f'Min TVL: {min_tvl}')
df = df[df['tvl'] >= min_tvl]
print(f'Protocols Tracked: {len(df)}')
# slugs = df['slug'].values.tolist()
# print(slugs[0])
# sevendayma, thirtydayma = [], []
# for slug in slugs:
#     dates, tvls = getProtocolTVLHistorical(slug)
#     tempa, tempb = processTVLData(dates, tvls)
#     sevendayma.append(tempa)
#     thirtydayma.append(tempb)
# df['7dma_%'] = sevendayma
# df['1mma_%'] = thirtydayma

Min TVL: 10000000
Protocols Tracked: 393


In [44]:
# volume stats
url = 'https://api.llama.fi/overview/dexs?excludeTotalDataChart=true&excludeTotalDataChartBreakdown=true&dataType=dailyVolume'
resp = requests.get(url)
resp = resp.json()


In [75]:
url = 'https://api.llama.fi/overview/dexs?excludeTotalDataChart=true&excludeTotalDataChartBreakdown=true&dataType=dailyVolume'
resp = requests.get(url)
resp = resp.json()
# print(df)
# print(resp['protocols'][0])
df_vol = pd.json_normalize(resp['protocols'])
# REname defillamJoin on 
df_vol = df_vol.rename(columns={'defillamaId': 'id'})
df_vol = df_vol[['id', 'change_7d', 'change_1m']]
# print(df_vol)
print(len(df_vol))

url = 'https://api.llama.fi/overview/options?excludeTotalDataChart=true&excludeTotalDataChartBreakdown=true&dataType=dailyPremiumVolume'
resp = requests.get(url).json()
# print(resp)
df_vol2 = pd.json_normalize(resp['protocols'])
df_vol2 = df_vol2.rename(columns={'defillamaId': 'id'})
df_vol2 = df_vol2[['id', 'change_7d', 'change_1m']]
# print(df_vol2)
print(len(df_vol2))
df_vol = pd.concat([df_vol, df_vol2]).reset_index(drop=True)
df_vol['id'].astype(int)
print(df_vol)


305
4
       id  change_7d  change_1m
0    2116     -30.21      90.84
1    2345      23.04      -5.02
2    2283     -55.23     -72.77
3     951     -63.44     -60.61
4    2695     -61.77     -74.42
..    ...        ...        ...
304  2198     -52.84     -36.88
305   128     -99.02     -98.30
306   503      42.96     -47.49
307   381      48.29     -62.22
308   534     -48.18      32.63

[309 rows x 3 columns]


In [95]:
df = df[['id', 'name', 'address', 'symbol']]
print(len(df))
df['id'].astype(int)
# print(df.info())
df['7dma_%'] = "7"
df['1mma_%'] = "30"
# print(df_vol.info())
df_vol['id'].astype(int)
print(len(df_vol))
df = df.merge(df_vol, how='left', on='id')
print(df['change_7d'].count())


393
309
81


In [4]:
a = []
if a:
    print('a')
else:
    print('false')

false


In [1]:
!pip install web3

                                              0.0/574.0 kB ? eta -:--:--
     --------------                         215.0/574.0 kB 4.5 MB/s eta 0:00:01
     -------------------------------------  573.4/574.0 kB 7.2 MB/s eta 0:00:01
     -------------------------------------- 574.0/574.0 kB 5.2 MB/s eta 0:00:00
  Using cached aiohttp-3.8.4-cp39-cp39-win_amd64.whl (323 kB)
  Using cached eth_abi-4.0.0-py3-none-any.whl (28 kB)
  Using cached eth_account-0.8.0-py3-none-any.whl (102 kB)
  Using cached eth_hash-0.5.1-py3-none-any.whl (9.0 kB)
  Using cached eth_typing-3.3.0-py3-none-any.whl (6.3 kB)
  Using cached eth_utils-2.1.0-py3-none-any.whl (24 kB)
  Using cached hexbytes-0.3.0-py3-none-any.whl (6.4 kB)
  Using cached jsonschema-4.17.3-py3-none-any.whl (90 kB)
  Using cached lru_dict-1.1.8-cp39-cp39-win_amd64.whl (12 kB)
                                              0.0/420.6 kB ? eta -:--:--
     ------------------------------------  419.8/420.6 kB 13.2 MB/s eta 0:00:01
     --------

In [2]:
# Holder Count Testing
import pandas as pd
from web3 import Web3

df = pd.read_csv('variant_db.csv')
print(len(df))
# print(len(df))
# print(df['address'].count())
# df['address'] = df['address'].fillna('No')
# df2 = df[~df["address"].str.contains(':')]
# df2 = df2[df2["address"] != 'No'].reset_index(drop=True)
# print(len(df2))
# print(df2)

hdc = df['address'].values.tolist()

# wallet_list = []
# print(len(hdc))
# print(Web3.is_address(hdc[0]), hdc[0])
# print(Web3.is_address(hdc[1]), hdc[1])

# for i, address in enumerate(hdc):
#     if Web3.is_address(address):
#     # if address == address and (":" not in address):
#         # print(f'searching: {address}')
#         # Get Holder Count Data (7d & 1m change)
#         wallet_list.append(address)


# wallet_list = [address for address in hdc if Web3.is_address(address)]
        

# wdf = pd.DataFrame(data={'wallet': wallet_list})
# wdf.to_csv('testaddresses.csv', index=False)
# print(wdf)

416


In [8]:
%%timeit # 10.4 ms ± 2.74 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
wallet_list = []
for address in hdc:
    if Web3.is_address(address):
        wallet_list.append(address)

# print(len(wallet_list))

6.38 ms ± 1.42 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
%%timeit #13 ms ± 3.61 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
eth_tokens = [address for address in hdc if Web3.is_address(address)]
# print(len(wallet_list2))

8.13 ms ± 2.58 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [31]:
import pandas as pd
import requests
from requests.auth import HTTPBasicAuth
from tqdm import tqdm
from web3 import Web3
from time import perf_counter, time
import datetime

# limit to 100k credits (2 credits per token holders request)
def getTokenHolderCount(tokenAddress, block=0):
    chainName = 'eth-mainnet'
    if block == 0:
        url = f"https://api.covalenthq.com/v1/{chainName}/tokens/{tokenAddress}/token_holders_v2/"
    else:
        url = f"https://api.covalenthq.com/v1/{chainName}/tokens/{tokenAddress}/token_holders_v2/?block-height={block}"
            
    headers = {
        "accept": "application/json",
    }

    basic = HTTPBasicAuth('cqt_rQVhYWhkHKYBPwPGYqQPKtDKbMCm', '')
    response = requests.get(url, headers=headers, auth=basic)
    resp = response.json()
    holder_count = resp.get('data', {}).get('pagination', {}).get('total_count', "NA")
    
    return holder_count


def getBlockByTimestamp(timestamp):
    # takes timestamp and returns block height
    m_apikey = "I42NRodUvq7iUeKVvs86RZZ7sFVYXvY9K1ZKrvzin4dJZK2aJC9GXYictplGAIpr"
    headers = {"X-API-Key": m_apikey, "accept": "application/json",}
    url = f'https://deep-index.moralis.io/api/v2/dateToBlock?chain=eth&date={timestamp}'
    resp = requests.get(url, headers=headers)
    results = resp.json()
    return results['block']

def getTokenHolderCountMetrics(tokens):
    # timestamps
    date_example = datetime.datetime.now(datetime.timezone.utc).replace(hour=0, minute=0, second=0, microsecond=0)
    ts_day = datetime.datetime.timestamp(date_example)
    ts = [ts_day, ts_day-(60*60*24*7), ts_day-(60*60*24*30)]
    blks = [getBlockByTimestamp(t) for t in ts]

    thcs_7day_dict, thcs_30day_dict = {}, {}
    for token in tokens:
        temp_thc = [getTokenHolderCount(token, blk) for blk in blks]
        day7 = (temp_thc[0]-temp_thc[1])/temp_thc[1]*100
        thcs_7day_dict[token] = day7
        day30 = (temp_thc[0]-temp_thc[2])/temp_thc[2]*100
        thcs_30day_dict[token] = day30

    thcs_7day = [thcs_7day_dict.get(token, 'NA') for token in tokens]
    thcs_30day = [thcs_30day_dict.get(token, 'NA') for token in tokens]
    
    return thcs_7day, thcs_30day


In [32]:
df = pd.read_csv('variant_db.csv')
hdc = df['address'].values.tolist()
eth_tokens = [address for address in hdc if Web3.is_address(address)]
eth_tokens = eth_tokens[0:2] # remove for production code
temp1, temp2 = getTokenHolderCountMetrics(eth_tokens) # remove for production code
# df['HolderCounts_7day%'], df['HolderCounts_30day%'] = getTokenHolderCountMetrics(eth_tokens)  # Uncomment for production



{'data': {'updated_at': '2023-05-05T14:09:00.872508673Z', 'chain_id': 1, 'chain_name': 'eth-mainnet', 'items': [{'contract_decimals': 18, 'contract_name': 'Lido DAO Token', 'contract_ticker_symbol': 'LDO', 'contract_address': '0x5a98fcbea516cf06857215779fd812ca3bef1b32', 'supports_erc': [], 'logo_url': 'https://logos.covalenthq.com/tokens/1/0x5a98fcbea516cf06857215779fd812ca3bef1b32.png', 'address': '0x3e40d73eb977dc6a537af587d48316fee66e9c8c', 'balance': '111182712058077181464024678', 'total_supply': '1000000000000000000000000000', 'block_height': 17190729}, {'contract_decimals': 18, 'contract_name': 'Lido DAO Token', 'contract_ticker_symbol': 'LDO', 'contract_address': '0x5a98fcbea516cf06857215779fd812ca3bef1b32', 'supports_erc': [], 'logo_url': 'https://logos.covalenthq.com/tokens/1/0x5a98fcbea516cf06857215779fd812ca3bef1b32.png', 'address': '0x09f82ccd6bae2aebe46ba7dd2cf08d87355ac430', 'balance': '70000000000000000000000000', 'total_supply': '1000000000000000000000000000', 'block_h

In [33]:
print(temp1)
print(temp2)

[0.13222950442681383, 0.06445545705258932]
[1.829981290926099, 0.8036617169620521]


In [14]:
# Troublingshooting 1
import pandas as pd
import requests
from requests.auth import HTTPBasicAuth
from tqdm import tqdm
from web3 import Web3
from time import perf_counter, sleep
from datetime import datetime, timezone


def getProtocols():
    resp = requests.get('https://api.llama.fi/protocols')
    return resp.json()


def getProtocolTVLHistorical(protocol_slug):
    url = f'https://api.llama.fi/protocol/{protocol_slug}'
    resp = requests.get(url)
    if resp.status_code == 200:
        resp = resp.json()
        temp_dates, temp_tvls = [], []
        for dict in resp['tvl']:
            temp_dates.append(dict['date'])
            temp_tvls.append(dict['totalLiquidityUSD'])
        return temp_dates, temp_tvls
    else:
        print(f'Attempting Retry: {protocol_slug}')
        resp = requests.get(url)
        if resp.status_code == 200:
            resp = resp.json()
            temp_dates, temp_tvls = [], []
            for dict in resp['tvl']:
                temp_dates.append(dict['date'])
                temp_tvls.append(dict['totalLiquidityUSD'])
            return temp_dates, temp_tvls
        else:
            print(f'Retry Failed: {protocol_slug}')
   

def processTVLData(tdates, ttvls):
    tdf = pd.DataFrame(data={'timestamp': tdates, 'tvls': ttvls})
    tdf['dates'] = tdf['timestamp'].values.astype(dtype='datetime64[s]')
    tdf['dates'] = pd.to_datetime(tdf['dates']).dt.date
    tdf['7dma'] = tdf['tvls'].rolling(window=7, min_periods=1).mean()
    tdf['7dma_%'] = tdf['7dma'].pct_change(periods=1).fillna(0)
    tdf['1mma'] = tdf['tvls'].rolling(window=30, min_periods=1).mean()
    tdf['1mma_%'] = tdf['1mma'].pct_change(periods=1).fillna(0)
    sevenday = tdf['7dma_%'].values.tolist()
    thirtyday = tdf['1mma_%'].values.tolist()
    return sevenday[-1], thirtyday[-1]


def getVolumeData():
    # Fetches DEX Volume Data
    url = 'https://api.llama.fi/overview/dexs?excludeTotalDataChart=true&excludeTotalDataChartBreakdown=true&dataType=dailyVolume'
    resp = requests.get(url).json()
    df_vol = pd.json_normalize(resp['protocols'])
    # df_vol = df_vol.rename(columns={'defillamaId': 'id', 'change_7d': 'Volume_7d', 'change_1m': 'Volume_1m'})
    df_vol = df_vol[['defillamaId', 'change_7d', 'change_1m']]
    # df_vol = df_vol[['id', 'Volume_7d', 'Volume_1m']]

    # Fetches Options Volume Data
    url = 'https://api.llama.fi/overview/options?excludeTotalDataChart=true&excludeTotalDataChartBreakdown=true&dataType=dailyPremiumVolume'
    resp = requests.get(url).json()
    df_vol2 = pd.json_normalize(resp['protocols'])
    # df_vol2 = df_vol2.rename(columns={'defillamaId': 'id', 'change_7d': 'Volume_7d_%', 'change_1m': 'Volume_1m_%'})
    df_vol2 = df_vol2[['defillamaId', 'change_7d', 'change_1m']]

    # Combines DEX & Option Volume Data to single Dataframe
    df_vol = pd.concat([df_vol, df_vol2]).reset_index(drop=True)
    df_vol = df_vol.rename(columns={'defillamaId': 'id', 'change_7d': 'Volume_7d_%', 'change_1m': 'Volume_1m_%'})
    df_vol['id'].astype(int)
    return df_vol


# limit to 100k credits (2 credits per token holders request)
def getTokenHolderCount(tokenAddress, block=0):
    # TODO: Upgrade Covalent request. URL input to function that makes request and handles all response status.
    try:
        chainName = 'eth-mainnet'
        if block == 0:
            url = f"https://api.covalenthq.com/v1/{chainName}/tokens/{tokenAddress}/token_holders_v2/"
        else:
            url = f"https://api.covalenthq.com/v1/{chainName}/tokens/{tokenAddress}/token_holders_v2/?block-height={block}"
                
        headers = {
            "accept": "application/json",
        }

        basic = HTTPBasicAuth('cqt_rQVhYWhkHKYBPwPGYqQPKtDKbMCm', '')
        response = requests.get(url, headers=headers, auth=basic)
        if response.status_code == 200:
            resp = response.json()
            holder_count = resp.get('data', {}).get('pagination', {}).get('total_count', "NA")
            if holder_count == None:
                return 0
            else:
                return holder_count
        else:
            print(f'Response Status Code: {response.status_code}')
            covalent_status_dict = {
            200: 'OK, Everything worked as expected.',
            400: 'Bad Request: The request could not be accepted, usually due to a missing required parameter.',
            401: 'Unauthorized:	No valid API key was provided.',
            404: 'Not Found: The request path does not exist.',
            429: 'Too Many Requests: You are being rate-limited. Please see the rate limiting section for more information.',
            500: "Server Errors: Something went wrong on Covalent's servers.",
            502: "Server Errors: Something went wrong on Covalent's servers.",
            503: "Server Errors: Something went wrong on Covalent's servers.",
            }
            print(f'Response Status Code = {response.status_code}: {covalent_status_dict[response.status_code]}')
            sleep(1)
    except TypeError as e:
        print('TypeError Occurred')
        print(f'tokenAddress: {tokenAddress}')
        print(f'block: {block}')


def getBlockByTimestamp(timestamp):
    # takes timestamp and returns block height
    m_apikey = "I42NRodUvq7iUeKVvs86RZZ7sFVYXvY9K1ZKrvzin4dJZK2aJC9GXYictplGAIpr"
    headers = {"X-API-Key": m_apikey, "accept": "application/json",}
    url = f'https://deep-index.moralis.io/api/v2/dateToBlock?chain=eth&date={timestamp}'
    resp = requests.get(url, headers=headers)
    results = resp.json()
    return int(results['block'])


def getTokenHolderCountMetrics(tokens):
    print('Fetching Token Holder Count Meitrcs')
    # generating daily, -7 day, -30 day timestamps
    date_example = datetime.now(timezone.utc).replace(hour=0, minute=0, second=0, microsecond=0)
    ts_day = datetime.timestamp(date_example)
    ts = [ts_day, ts_day-(60*60*24*7), ts_day-(60*60*24*30)]
    # inputs timestamp and returns closest block number for the given timestamp
    blks = [getBlockByTimestamp(t) for t in ts]
    print(blks)

    # generates lookup dictionary for tokens
    thcs_7day_dict, thcs_30day_dict = {}, {}
    try:
        for token in tqdm(tokens):
            start = perf_counter()
            temp_thc = [getTokenHolderCount(token, blk) for blk in blks]
            

            day7 = (temp_thc[0]-temp_thc[1])/temp_thc[1]*100
            thcs_7day_dict[token] = day7
            day30 = (temp_thc[0]-temp_thc[2])/temp_thc[2]*100
            
            thcs_30day_dict[token] = day30
            end = perf_counter()
            # Rate limiter for API call of ~4-5 requests/sec
            if (end-start) < 1.5:
                sleep(1.5-(end-start))
    finally:
        print(f'token: {token}')
        print(f'Blocks: {blks}')
        print(f'temp_thc: {temp_thc}')

    thcs_7day = [thcs_7day_dict.get(token, 'NA') for token in tokens]
    thcs_30day = [thcs_30day_dict.get(token, 'NA') for token in tokens]
    
    return thcs_7day, thcs_30day


start_time = perf_counter()

# Fetches all protocols on Defi Llama
protocols = getProtocols()

# Initializes Dataframe
df = pd.json_normalize(protocols)

# Filters Protocols based on minimum tvl requirement
min_mcap = 100_000_000
min_tvl = 10_000_000
print(f'Min TVL: {min_tvl} | Min mcap: {min_mcap}')
df = df[(df['tvl'] >= min_tvl) | (df['mcap'] >= min_mcap)]
print(f'Protocols Tracked: {len(df)}')

# Reduces Dataframe to only required Columns
df = df[['id', 'category', 'name', 'address', 'symbol', 'tvl', 'mcap', 'slug']]
df['id'].astype(int)

# Creates List of Protocal names
slugs = df['slug'].values.tolist()
df.to_csv('variant_db_ts1.csv', index=False)

Min TVL: 10000000 | Min mcap: 100000000
Protocols Tracked: 415


In [15]:
hdc = df['address'].values.tolist()
eth_tokens = [address for address in hdc if Web3.is_address(address)]
chain_stats = ["ETH" if Web3.is_address(address) else "TBD" for address in hdc]

thcs_7day_dict, thcs_30day_dict = getTokenHolderCountMetrics(eth_tokens)

df['HolderCounts_7day%'] = [thcs_7day_dict.get(token, 'NA') for token in hdc]
df['HolderCounts_30day%'] = [thcs_30day_dict.get(token, 'NA') for token in hdc]

Fetching Token Holder Count Meitrcs
[17197858, 17148050, 16985956]
<class 'int'> <class 'int'> <class 'int'>


 71%|███████   | 108/153 [06:20<02:38,  3.52s/it]

token: 0xF1182229B71E79E504b1d2bF076C15a277311e05
Blocks: [17197858, 17148050, 16985956]
temp_thc: [369, None, None]


TypeError: unsupported operand type(s) for -: 'int' and 'NoneType'

In [19]:
dd = [369, None, None]

def checking(da):
    if da == None:
        print('holders 0')
        return 0
    else:
        print('calcing')
        return da
    
for d in dd:
    a = checking(d)
    print(a)

calcing
369
holders 0
0
holders 0
0
